In [2]:
import pickle
import numpy as np

In [26]:
X = pickle.load(open('A.pkl','rb'))
y = pickle.load(open('B.pkl','rb'))
ssf_arrays = pickle.load(open('SSF_arrays.pkl','rb'))

In [24]:
ssf_array = np.array(ssf_arrays)
y

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_3')>

In [16]:
ssf_array.shape

(932, 3)

In [17]:
X = X/255.0

In [18]:
X

array([[[[0.00392157, 0.04313725, 0.1372549 ],
         [0.00784314, 0.03921569, 0.11764706],
         [0.00784314, 0.03529412, 0.10980392],
         ...,
         [0.        , 0.01960784, 0.03137255],
         [0.00392157, 0.01960784, 0.03921569],
         [0.        , 0.02352941, 0.05098039]],

        [[0.        , 0.03529412, 0.09019608],
         [0.01568627, 0.05098039, 0.12941176],
         [0.00784314, 0.03921569, 0.11764706],
         ...,
         [0.        , 0.01568627, 0.02745098],
         [0.00392157, 0.02352941, 0.04313725],
         [0.        , 0.03529412, 0.0627451 ]],

        [[0.00392157, 0.04313725, 0.05882353],
         [0.        , 0.03921569, 0.11372549],
         [0.01568627, 0.05882353, 0.14117647],
         ...,
         [0.        , 0.01568627, 0.02745098],
         [0.01176471, 0.04313725, 0.0627451 ],
         [0.00784314, 0.05490196, 0.07843137]],

        ...,

        [[0.11372549, 0.1372549 , 0.16470588],
         [0.00392157, 0.02745098, 0.05098039]

In [19]:
ssf_array

array([[3.09723438e+03, 2.00000000e+00, 5.00000000e+00],
       [2.89284660e+03, 0.00000000e+00, 0.00000000e+00],
       [3.49985206e+03, 3.00000000e+00, 6.00000000e+00],
       ...,
       [3.57000000e+01, 2.00000000e+00, 2.00000000e+00],
       [1.91018584e+03, 7.00000000e+00, 9.00000000e+00],
       [4.62762358e+01, 1.00000000e+00, 1.00000000e+00]])

In [20]:
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, concatenate, Input

In [21]:
input_img = Input(shape=(150,150,3))
x = Conv2D(64, (3,3), activation='relu')(input_img)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
image_model = Model(inputs=input_img, outputs=x)

input_int = Input(shape=(3,))
y = Dense(128, activation='relu')(input_int)
ssf_model = Model(inputs=input_int, outputs=y)

merged = concatenate([image_model.output, ssf_model.output])
merged = Dense(128, activation='relu')(merged)
merged = Dense(2, activation='softmax')(merged)

model = Model(inputs=[image_model.input, ssf_model.input], outputs=merged)

In [22]:
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
y = np.array(y)
model.fit([X,ssf_array],y, epochs=10, validation_data=([X,ssf_array],y))

Epoch 1/10
30/30 [==============================] - 76s 2s/step - loss: 420.3996 - accuracy: 0.5313 - val_loss: 1.0900 - val_accuracy: 0.6019
Epoch 2/10
30/30 [==============================] - 45s 1s/step - loss: 1.6173 - accuracy: 0.7047 - val_loss: 0.4043 - val_accuracy: 0.8230
Epoch 3/10
30/30 [==============================] - 44s 1s/step - loss: 0.4046 - accuracy: 0.8455 - val_loss: 0.3723 - val_accuracy: 0.8423
Epoch 4/10
30/30 [==============================] - 44s 1s/step - loss: 0.7381 - accuracy: 0.8072 - val_loss: 0.2832 - val_accuracy: 0.8841
Epoch 5/10
30/30 [==============================] - 46s 2s/step - loss: 0.8067 - accuracy: 0.8250 - val_loss: 0.3069 - val_accuracy: 0.8970
Epoch 6/10
30/30 [==============================] - 45s 2s/step - loss: 0.2608 - accuracy: 0.8961 - val_loss: 0.1755 - val_accuracy: 0.9335
Epoch 7/10
30/30 [==============================] - 50s 2s/step - loss: 0.6252 - accuracy: 0.8863 - val_loss: 1.1598 - val_accuracy: 0.7189
Epoch 8/10
30/30 [

In [28]:
model.save('64x2-CNN_impure_ssf.model')

INFO:tensorflow:Assets written to: 64x2-CNN_impure_ssf.model\assets
